In [ ]:
%load_ext autoreload
%autoreload 2

import evolution
from config import Config
from pymoo.algorithms.moo.nsga2 import NSGA2
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

config = Config()
problem = evolution.EVProblem(config)
algorithm = NSGA2(pop_size=config.pop_size,
                    sampling=evolution.SamplingAll(),
                    mutation=evolution.MutationAll(),
                    eliminate_duplicates=True)

In [ ]:
from pymoo.optimize import minimize

res = minimize(problem, 
                algorithm,
                callback=evolution.do_every_generations, 
                termination=config.termination)

In [3]:
import numpy as np

train_labels = np.array([])
train_labels2 = np.array([0, 1])
# np.concatenate(train_labels, train_labels2)
np.concatenate((train_labels, train_labels2), axis=0)

array([0., 1.])

In [ ]:

train_labels = np.array([0])
for batch in train_labels_batched:
    print(train_labels)
    # print(batch.numpy())
    train_labels = np.concatenate((train_labels, batch.numpy()), axis=0)
    # train_labels = np.concatenate(train_labels, train_labels2)